In [1]:
import os
import numpy as np
import xarray as xr
from xhistogram.xarray import histogram
from cmocean import cm

from matplotlib import pyplot as plt
%matplotlib inline

import mitequinox.utils as ut
import mitequinox.plot as pl

In [2]:
from dask.distributed import Client, LocalCluster
#
#cluster = LocalCluster()
#
from dask_jobqueue import PBSCluster
cluster = PBSCluster() #processes=7, cores=7
w = cluster.scale(jobs=10)
#
client = Client(cluster)



In [3]:
client

Client Scheduler: tcp://10.148.1.44:60094 Dashboard: http://10.148.1.44:8787/status,Cluster Workers: 70 Cores: 70 Memory: 1.09 TiB


In [4]:
# same but over all variables and faces
V = ["SSU","SSV"]
#V = V+["KPPhbl", "qsoceTAUX", "oceTAUY"]

In [5]:
def compute_store(v, ds, diag, op):
    #i, j = ut.get_ij_dims(ds)
    diag_zarr = diag+"_"+v
    if not ut.is_diagnostic(diag_zarr):
        out, tmp_zarr = ut.custom_distribute(ds, op, 
                                             tmp_dir=ut.scratch+"tmp/", 
                                             face=1, ) # **{j: 1080}
        ut.store_diagnostic(diag_zarr, out, overwrite=True)
        print("{} {} stored".format(v, diag))
    else:
        print("{} {} already exists".format(v, diag))

In [15]:
diags = {"prime_mean_sq": lambda x: ((x-x.mean("time"))**2).mean("time")
        #, "mean_square": lambda x: (x**2).mean("time"),
        }
        
for v in V:
    ds = ut.load_data(v)
    for diag, op in diags.items():
        compute_store(v, ds, diag, op)

data stored in /home/datawork-lops-osi/equinox/mit4320/diags/prime_mean_sq_SSU.zarr
SSU prime_mean_sq stored
data stored in /home/datawork-lops-osi/equinox/mit4320/diags/prime_mean_sq_SSV.zarr
SSV prime_mean_sq stored


In [6]:
ds_tot = ut.load_data(V)

In [7]:
sli= 2
grid = xr.open_zarr('/home1/datawork/xyu/grid/mit_grid.zarr')
ds_tot = xr.merge([ds_tot,grid]).isel(i_g=slice(0,None,sli),
                                      j_g=slice(0,None,sli),i=slice(0,None,sli),j=slice(0,None,sli))

In [8]:
ds_tot

<xarray.Dataset>
Dimensions:  (face: 13, i: 2160, i_g: 2160, j: 2160, j_g: 2160, time: 8785)
Coordinates: (12/22)
  * face     (face) int64 0 1 2 3 4 5 6 7 8 9 10 11 12
  * i_g      (i_g) int64 0 2 4 6 8 10 12 ... 4306 4308 4310 4312 4314 4316 4318
  * j        (j) int64 0 2 4 6 8 10 12 14 ... 4306 4308 4310 4312 4314 4316 4318
    niter    (time) int64 dask.array<chunksize=(200,), meta=np.ndarray>
  * time     (time) datetime64[ns] 2011-11-15 2011-11-15T01:00:00 ... 2012-11-15
  * i        (i) int64 0 2 4 6 8 10 12 14 ... 4306 4308 4310 4312 4314 4316 4318
    ...       ...
    dyC      (face, j_g, i) float32 dask.array<chunksize=(1, 2160, 2160), meta=np.ndarray>
    dyG      (face, j, i_g) float32 dask.array<chunksize=(1, 2160, 2160), meta=np.ndarray>
    rA       (face, j, i) float32 dask.array<chunksize=(1, 2160, 2160), meta=np.ndarray>
    rAs      (face, j_g, i) float32 dask.array<chunksize=(1, 2160, 2160), meta=np.ndarray>
    rAw      (face, j, i_g) float32 dask.array<chunksize=(1, 2160, 2160), meta=np.ndarray>
    rAz      (face, j_g, i_g) float32 dask.array<chunksize=(1, 2160, 2160), meta=np.ndarray>
Data variables:
    SSU      (time, face, j, i_g) float32 dask.array<chunksize=(1, 1, 2160, 2160), meta=np.ndarray>
    SSV      (time, face, j_g, i) float32 dask.array<chunksize=(1, 1, 2160, 2160), meta=np.ndarray>

In [8]:
# Rotation velocity field
ds_tot['U'] =  ds_tot.SSU.rename({'i_g': 'i'})*ds_tot.CS - ds_tot.SSV.rename({'j_g': 'j'})*ds_tot.SN
ds_tot['V'] =  ds_tot.SSU.rename({'i_g': 'i'})*ds_tot.SN + ds_tot.SSV.rename({'j_g': 'j'})*ds_tot.CS

ds_tot['lon'] = 0*ds_tot.face+ds_tot.XC
ds_tot['lat'] = 0*ds_tot.face+ds_tot.YC

#V['lon'] = 0*ds_tot.face+ds_tot.XC
#V['lat'] = 0*ds_tot.face+ds_tot.YC

In [11]:
ds_tot = ds_tot.persist()

distributed.core - ERROR - 'tcp://10.148.0.71:47275'
Traceback (most recent call last):
  File "/home1/datahome/zcaspar/miniconda3/envs/croco/lib/python3.8/site-packages/distributed/core.py", line 573, in handle_stream
    handler(**merge(extra, msg))
  File "/home1/datahome/zcaspar/miniconda3/envs/croco/lib/python3.8/site-packages/distributed/scheduler.py", line 5018, in handle_release_data
    ws: WorkerState = parent._workers_dv[worker]
KeyError: 'tcp://10.148.0.71:47275'
distributed.utils - ERROR - 'tcp://10.148.0.71:47275'
Traceback (most recent call last):
  File "/home1/datahome/zcaspar/miniconda3/envs/croco/lib/python3.8/site-packages/distributed/utils.py", line 668, in log_errors
    yield
  File "/home1/datahome/zcaspar/miniconda3/envs/croco/lib/python3.8/site-packages/distributed/scheduler.py", line 4032, in add_worker
    await self.handle_worker(comm=comm, worker=address)
  File "/home1/datahome/zcaspar/miniconda3/envs/croco/lib/python3.8/site-packages/distributed/schedule

In [9]:
ds_tot = ds_tot.drop(['i_g','j_g','SSU','SSV'])

In [10]:
ds_tot['KE'] = ds_tot['U']**2+ds_tot['V']**2

In [15]:
ds_tot = ds_tot.drop(['U','V'])

In [16]:
ds_tot

<xarray.Dataset>
Dimensions:  (face: 13, i: 144, i_g: 144, j: 144, j_g: 144, time: 8785)
Coordinates: (12/20)
  * face     (face) int64 0 1 2 3 4 5 6 7 8 9 10 11 12
  * j        (j) int64 0 30 60 90 120 150 180 ... 4140 4170 4200 4230 4260 4290
    niter    (time) int64 dask.array<chunksize=(200,), meta=np.ndarray>
  * time     (time) datetime64[ns] 2011-11-15 2011-11-15T01:00:00 ... 2012-11-15
  * i        (i) int64 0 30 60 90 120 150 180 ... 4140 4170 4200 4230 4260 4290
    CS       (face, j, i) float32 dask.array<chunksize=(1, 144, 144), meta=np.ndarray>
    ...       ...
    dyC      (face, j_g, i) float32 dask.array<chunksize=(1, 144, 144), meta=np.ndarray>
    dyG      (face, j, i_g) float32 dask.array<chunksize=(1, 144, 144), meta=np.ndarray>
    rA       (face, j, i) float32 dask.array<chunksize=(1, 144, 144), meta=np.ndarray>
    rAs      (face, j_g, i) float32 dask.array<chunksize=(1, 144, 144), meta=np.ndarray>
    rAw      (face, j, i_g) float32 dask.array<chunksize=(1, 144, 144), meta=np.ndarray>
    rAz      (face, j_g, i_g) float32 dask.array<chunksize=(1, 144, 144), meta=np.ndarray>
Dimensions without coordinates: i_g, j_g
Data variables:
    lon      (face, j, i) float64 dask.array<chunksize=(1, 144, 144), meta=np.ndarray>
    lat      (face, j, i) float64 dask.array<chunksize=(1, 144, 144), meta=np.ndarray>
    KE       (time, face, j, i) float32 dask.array<chunksize=(1, 1, 144, 144), meta=np.ndarray>

In [13]:
diags = {"mean": lambda x: x.mean("time")
        #, "mean_square": lambda x: (x**2).mean("time"),
        }
V = ['KE']        
for v in V:
    for diag, op in diags.items():
        compute_store(v, ds_tot, diag, op)

distributed.core - ERROR - 'tcp://10.148.0.111:52292'
Traceback (most recent call last):
  File "/home1/datahome/zcaspar/miniconda3/envs/croco/lib/python3.8/site-packages/distributed/core.py", line 573, in handle_stream
    handler(**merge(extra, msg))
  File "/home1/datahome/zcaspar/miniconda3/envs/croco/lib/python3.8/site-packages/distributed/scheduler.py", line 5018, in handle_release_data
    ws: WorkerState = parent._workers_dv[worker]
KeyError: 'tcp://10.148.0.111:52292'
distributed.utils - ERROR - 'tcp://10.148.0.111:52292'
Traceback (most recent call last):
  File "/home1/datahome/zcaspar/miniconda3/envs/croco/lib/python3.8/site-packages/distributed/utils.py", line 668, in log_errors
    yield
  File "/home1/datahome/zcaspar/miniconda3/envs/croco/lib/python3.8/site-packages/distributed/scheduler.py", line 4032, in add_worker
    await self.handle_worker(comm=comm, worker=address)
  File "/home1/datahome/zcaspar/miniconda3/envs/croco/lib/python3.8/site-packages/distributed/sched

Traceback (most recent call last):
  File "/home1/datahome/zcaspar/miniconda3/envs/croco/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3441, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-13-321037e94a9c>", line 7, in <module>
    compute_store(v, ds_tot, diag, op)
  File "<ipython-input-5-a31ca17c6b9d>", line 5, in compute_store
    out, tmp_zarr = ut.custom_distribute(ds, op,
  File "/home1/datahome/zcaspar/mit_equinox/mitequinox/utils.py", line 506, in custom_distribute
    out.to_zarr(zarr, mode="w")
  File "/home1/datahome/zcaspar/miniconda3/envs/croco/lib/python3.8/site-packages/xarray/core/dataset.py", line 1790, in to_zarr
    return to_zarr(
  File "/home1/datahome/zcaspar/miniconda3/envs/croco/lib/python3.8/site-packages/xarray/backends/api.py", line 1476, in to_zarr
    writes = writer.sync(compute=compute)
  File "/home1/datahome/zcaspar/miniconda3/envs/croco/lib/python3.8/site-packages/xarray/backends/common.p

TypeError: object of type 'NoneType' has no len()

In [1]:
cluster.close()

NameError: name 'cluster' is not defined